In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas
from PIL import Image
from datetime import datetime
import requests
import os
import openpyxl
import re

# Descobre automaticamente o diretório onde está o script (.py)
if "__file__" in globals():
    # Quando está rodando como script normal (python meu_script.py)
    script_dir = os.path.dirname(os.path.abspath(__file__))
else:
    # Quando está rodando em ambiente interativo (IDLE, Jupyter, etc.)
    script_dir = os.getcwd()

print("Diretório do script:", script_dir)  # Apenas para conferência

# Define os caminhos dos arquivos com base no diretório do script
LAST_PATH_FILE = os.path.join(script_dir, "last_path.txt")
EXCEL_FILE = os.path.join(script_dir, "dados_cotacoes.xlsx")
COTACAO_SEQUENCE_FILE = os.path.join(script_dir, "cotacao_sequence.txt")

# Linha base para o primeiro item (por causa das linhas de cabeçalho)
BASE_ITEM_ROW = 5


# Função para obter o número sequencial atual e incrementá-lo
def obter_numero_sequencial(filepath=COTACAO_SEQUENCE_FILE):
    try:
        with open(filepath, "r", encoding="utf-8") as file:
            numero_atual = int(file.read().strip())
    except FileNotFoundError:
        numero_atual = 3456  # Número inicial se o arquivo não existir

    novo_numero = numero_atual + 1

    with open(filepath, "w", encoding="utf-8") as file:
        file.write(str(novo_numero))

    return numero_atual


# Função para buscar dados do cliente a partir do CNPJ (usando ReceitaWS)
def buscar_dados_cliente(cnpj):
    # Remove tudo que não for número
    cnpj = re.sub(r"\D", "", cnpj)

    # Valida tamanho
    if len(cnpj) != 14:
        messagebox.showerror("Erro", "CNPJ inválido. Digite 14 dígitos (com ou sem pontuação).")
        return None

    url = f"https://www.receitaws.com.br/v1/cnpj/{cnpj}"

    try:
        response = requests.get(url, timeout=10)
    except requests.RequestException as e:
        messagebox.showerror("Erro", f"Falha ao conectar à API da ReceitaWS:\n{e}")
        return None

    if response.status_code != 200:
        # Tenta extrair mensagem de erro mais amigável
        try:
            dados_erro = response.json()
            msg = dados_erro.get("message", str(dados_erro))
        except Exception:
            msg = response.text

        messagebox.showerror(
            "Erro",
            f"Erro na consulta à API (HTTP {response.status_code}):\n{msg}",
        )
        return None

    # Agora tenta interpretar o JSON
    try:
        dados = response.json()
    except ValueError:
        messagebox.showerror("Erro", "Resposta inválida da API (não é JSON).")
        return None

    # A API da ReceitaWS geralmente envia um campo "status" com "OK" ou "ERROR"
    if dados.get("status") != "OK":
        message = dados.get("message", "Erro desconhecido na API da ReceitaWS.")
        messagebox.showerror("Erro na consulta ao CNPJ", message)
        return None

    endereco = f'{dados.get("logradouro", "")}, {dados.get("numero", "")} - {dados.get("bairro", "")}'
    cidade_uf_cep = f'{dados.get("municipio", "")} - {dados.get("uf", "")}, CEP: {dados.get("cep", "")}'
    telefone = dados.get("telefone", "")

    return {
        "razao_social": dados.get("nome", ""),
        "endereco": endereco,
        "cidade_uf_cep": cidade_uf_cep,
        "telefone": telefone,
    }


# Função para quebrar texto em múltiplas linhas com limite de caracteres
def wrap_text(text, char_limit=85):
    words = text.split()
    lines = []
    line = []
    for word in words:
        if len(" ".join(line + [word])) > char_limit:
            lines.append(" ".join(line))
            line = [word]
        else:
            line.append(word)
    if line:
        lines.append(" ".join(line))
    return lines


# Função para gerar o PDF da cotação
def gerar_cotacao_pdf(dados_cliente, itens, prazo_pagamento, frete_tipo,
                      output_path, numero_sequencial, responsavel, empresa):
    if empresa == "Mokka":
        logo_filename = "Logo-Mokka-Sensors.jpg"
        footer_text = "Mokka Com. de Bens de Consumo Ltda - CNPJ: 21.220.932/0001-10"
        logo_percent = 0.35  # Tamanho do logo para Mokka
        logo_y_position_adjustment = 2 * cm
    else:
        logo_filename = "Logo-Moica.jpg"
        footer_text = "Moica Com. de Bens de Consumo Ltda - CNPJ: 42.044.083/0001-60"
        logo_percent = 0.11  # Tamanho do logo para Moica
        logo_y_position_adjustment = 0.35 * cm

    # Caminho completo do logo
    logo_path = os.path.join(script_dir, logo_filename)

    # Verifica se o arquivo de logo existe
    if not os.path.exists(logo_path):
        messagebox.showerror("Erro", f"O arquivo de logotipo não foi encontrado: {logo_path}")
        return

    logo_image = Image.open(logo_path)
    original_width, original_height = logo_image.size

    # Dimensões da página
    width, height = A4

    # Calcula novo tamanho do logo
    desired_width = width * logo_percent
    aspect_ratio = original_height / original_width
    desired_height = desired_width * aspect_ratio

    # Cria o canvas do PDF
    c = canvas.Canvas(output_path, pagesize=A4)
    page_number = 1

    # Função para desenhar cabeçalho em todas as páginas
    def draw_header(cnv, page_num):
        logo_y_position = height - logo_y_position_adjustment - desired_height
        cnv.drawImage(logo_path, 2 * cm, logo_y_position,
                      width=desired_width, height=desired_height)

        line_y_position = logo_y_position - 0.3 * cm
        cnv.setLineWidth(0.5)
        cnv.line(1 * cm, line_y_position, width - 1 * cm, line_y_position)

        data_criacao = datetime.now().strftime("%d/%m/%Y")
        cnv.setFont("Helvetica", 12)
        cnv.drawString(width - 5 * cm, height - 3 * cm, f"Data: {data_criacao}")

        cnv.setFont("Helvetica", 10)
        cnv.drawString(2 * cm, height - 4 * cm, f"Responsável: {responsavel['nome']}")
        cnv.drawString(2 * cm, height - 4.5 * cm, f"Contato: {responsavel['telefone']}")
        cnv.drawString(2 * cm, height - 5 * cm, f"{responsavel['email']}")
        cnv.drawString(2 * cm, height - 5.5 * cm, f"{responsavel['site1']}")
        cnv.drawString(2 * cm, height - 6 * cm, f"{responsavel['site2']}")

        footer_y_position = 1.5 * cm
        cnv.setLineWidth(0.5)
        cnv.line(1 * cm, footer_y_position + 0.5 * cm,
                 width - 1 * cm, footer_y_position + 0.5 * cm)
        cnv.setFont("Helvetica", 10)
        cnv.drawCentredString(width / 2, footer_y_position, footer_text)
        cnv.drawString(width - 5 * cm, footer_y_position, f"Página {page_num}")

    draw_header(c, page_number)

    # Título com número sequencial
    c.setFont("Helvetica-Bold", 16)
    c.drawString(2 * cm, height - 7 * cm, f"Cotação #{numero_sequencial}")

    # Subtítulo Cliente
    c.setFont("Helvetica-Bold", 14)
    c.drawString(2 * cm, height - 8.5 * cm, "Cliente")

    # Dados do cliente (com campos opcionais)
    c.setFont("Helvetica", 12)
    client_y = height - 9.5 * cm
    c.drawString(2 * cm, client_y, f"Razão Social: {dados_cliente.get('razao_social', '')}")

    endereco = dados_cliente.get("endereco", "")
    cidade_uf_cep = dados_cliente.get("cidade_uf_cep", "")
    telefone = dados_cliente.get("telefone", "")

    if endereco:
        client_y -= 0.5 * cm
        c.drawString(2 * cm, client_y, f"Endereço: {endereco}")

    if cidade_uf_cep:
        client_y -= 0.5 * cm
        c.drawString(2 * cm, client_y, f"{cidade_uf_cep}")

    if telefone:
        client_y -= 0.5 * cm
        c.drawString(2 * cm, client_y, f"Telefone: {telefone}")

    client_y -= 0.5 * cm
    c.setLineWidth(0.5)
    c.line(1 * cm, client_y, width - 1 * cm, client_y)

    current_y = height - 13 * cm

    for i, item in enumerate(itens, start=1):
        if current_y < 5 * cm:  # Margem para quebra de página
            c.showPage()
            page_number += 1
            draw_header(c, page_number)
            current_y = height - 7 * cm

        # Subtítulo do item
        c.setFont("Helvetica-Bold", 14)
        c.drawString(2 * cm, current_y, f"Item {i}")
        current_y -= 1 * cm

        # Descrição do produto
        c.setFont("Helvetica", 12)
        c.drawString(2 * cm, current_y, f"Produto: {item['produto']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, "Descrição:")
        current_y -= 0.5 * cm

        # Descrição detalhada (quebra de linhas)
        descricao_lines = wrap_text(item["descricao"], char_limit=80)
        c.setFont("Helvetica", 12)
        for line in descricao_lines:
            if current_y < 2.5 * cm:
                c.showPage()
                page_number += 1
                draw_header(c, page_number)
                current_y = height - 7 * cm
                c.setFont("Helvetica", 12)
            c.drawString(2 * cm, current_y, line)
            current_y -= 0.6 * cm

        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y,
                     f"Preço Unitário: R$ {item['preco_unitario'].replace('.', ',')}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"Quantidade: {item['quantidade']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y, f"NCM: {item['ncm']}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y,
                     f"Total: R$ {item['total'].replace('.', ',')}")
        current_y -= 0.5 * cm
        c.drawString(2 * cm, current_y,
                     f"Prazo de Entrega: {item['prazo_entrega']}")
        current_y -= 2 * cm

    if current_y < 5 * cm:
        c.showPage()
        page_number += 1
        draw_header(c, page_number)
        current_y = height - 7 * cm

    # Total da cotação
    total_cotacao = sum(float(item["total"].replace(",", ".")) for item in itens)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(2 * cm, current_y,
                 f"TOTAL DA COTAÇÃO: R$ {total_cotacao:.2f}".replace(".", ","))
    current_y -= 1 * cm

    # Prazo de pagamento
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, current_y, f"Prazo de Pagamento: {prazo_pagamento}")
    current_y -= 1 * cm

    # Frete
    c.setFont("Helvetica", 12)
    c.drawString(2 * cm, current_y, f"Frete: {frete_tipo}")
    current_y -= 1 * cm

    # Rodapé da página atual
    footer_y_position = 1.5 * cm
    c.setLineWidth(0.5)
    c.line(1 * cm, footer_y_position + 0.5 * cm,
           width - 1 * cm, footer_y_position + 0.5 * cm)
    c.setFont("Helvetica", 10)
    c.drawCentredString(width / 2, footer_y_position, footer_text)
    c.drawString(width - 5 * cm, footer_y_position, f"Página {page_number}")

    # Página com condições de fornecimento
    c.showPage()
    page_number += 1
    draw_header(c, page_number)
    current_y = height - 9 * cm

    c.setFont("Helvetica-Bold", 14)
    c.drawString(2 * cm, current_y, "Condições de fornecimento")
    current_y -= 1 * cm

    c.setFont("Helvetica", 10)
    c.drawString(2 * cm, current_y, "1. A cotação é válida por 5 dias.")
    current_y -= 1 * cm

    c.drawString(2 * cm, current_y, "2. O prazo de entrega está sujeito a alterações.")
    current_y -= 1 * cm

    c.drawString(2 * cm, current_y, "3. Se o item não estiver disponível para pronta entrega, ele será fabricado ou importado especificamente para")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "atender à sua necessidade. Nesses casos, não aceitamos devoluções ou cancelamentos de pedidos.")
    current_y -= 1 * cm

    c.drawString(2 * cm, current_y, "4. Nossa empresa é optante pelo Simples Nacional, um regime tributário diferenciado e simplificado. Neste regime,")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "os impostos são pagos de forma unificada e estão todos inclusos no preço dos nossos produtos. Isso inclui tributos")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "federais, estaduais e municipais, como o Imposto de Renda, CSLL, PIS, COFINS, ICMS e ISS. Empresas que")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "compram nossos produtos para revenda ou industrialização podem gerar crédito de ICMS com uma alíquota de")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "aproximadamente 4%. Esse crédito pode ser usado para compensar o ICMS em operações futuras.")
    current_y -= 1 * cm

    # Novo item 5
    c.drawString(2 * cm, current_y, "5. Nos casos em que oferecemos um item similar ao originalmente solicitado pelo cliente, realizamos uma análise")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "técnica cuidadosa para garantir que o item proposto seja adequado. No entanto, é de responsabilidade exclusiva")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "do cliente realizar uma verificação detalhada e assegurar que o item proposto atenda plenamente às suas")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "necessidades específicas.")
    current_y -= 1 * cm

    # Novo item 6
    c.drawString(2 * cm, current_y, "6. O faturamento a prazo está sujeito a uma análise financeira, a qual será realizada somente após o recebimento")
    current_y -= 0.5 * cm
    c.drawString(2 * cm, current_y, "do pedido de compra.")

    footer_y_position = 1.5 * cm
    c.setLineWidth(0.5)
    c.line(1 * cm, footer_y_position + 0.5 * cm,
           width - 1 * cm, footer_y_position + 0.5 * cm)
    c.setFont("Helvetica", 10)
    c.drawCentredString(width / 2, footer_y_position, footer_text)
    c.drawString(width - 5 * cm, footer_y_position, f"Página {page_number}")

    c.save()


# Função para adicionar dados ao Excel
def adicionar_dados_excel(dados_cliente, itens, numero_sequencial, cliente_tipo):
    if not os.path.exists(EXCEL_FILE):
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(
            [
                "Data da Cotação",
                "Número da Cotação",
                "Razão Social do Cliente",
                "CNPJ do Cliente",
                "Produto",
                "Total (item)",
                "Cliente Final ou Revenda",
            ]
        )
    else:
        wb = openpyxl.load_workbook(EXCEL_FILE)
        ws = wb.active

    data_cotacao = datetime.now().strftime("%d/%m/%Y")

    # Se estiver em modo API, pegamos o CNPJ; se for manual, deixamos em branco
    modo = cliente_mode_var.get() if "cliente_mode_var" in globals() else "api"
    if modo == "api":
        cnpj_cliente = entrada_cnpj.get()
    else:
        cnpj_cliente = ""

    for item in itens:
        ws.append(
            [
                data_cotacao,
                numero_sequencial,
                dados_cliente["razao_social"],
                cnpj_cliente,
                item["produto"],
                item["total"].replace(".", ","),
                cliente_tipo,
            ]
        )

    wb.save(EXCEL_FILE)


# Funções para o caminho do último PDF
def obter_ultimo_caminho():
    if os.path.exists(LAST_PATH_FILE):
        with open(LAST_PATH_FILE, "r", encoding="utf-8") as file:
            return file.read().strip()
    return ""


def salvar_ultimo_caminho(caminho):
    with open(LAST_PATH_FILE, "w", encoding="utf-8") as file:
        file.write(caminho)


def obter_caminho_saida(numero_sequencial):
    initial_dir = obter_ultimo_caminho() or os.getcwd()
    output_path = filedialog.asksaveasfilename(
        defaultextension=".pdf",
        title="Salvar Cotação Como",
        initialfile=f"Cotação {empresa_var.get()} {numero_sequencial}.pdf",
        filetypes=[("PDF files", "*.pdf")],
        initialdir=initial_dir,
    )
    if output_path:
        salvar_ultimo_caminho(os.path.dirname(output_path))
    return output_path


# Validações
def validate_decimal(P):
    if P == "":
        return True
    try:
        float(P.replace(",", "."))
        return True
    except ValueError:
        return False


def validate_integer(P):
    if P == "":
        return True
    try:
        int(P)
        return True
    except ValueError:
        return False


# Cálculo de totais
def calcular_total(item_frame):
    try:
        preco_unitario = float(item_frame["preco_unitario"].get().replace(",", "."))
        quantidade = int(item_frame["quantidade"].get())
        total = preco_unitario * quantidade
        item_frame["total"].config(state="normal")
        item_frame["total"].delete(0, tk.END)
        item_frame["total"].insert(0, f"{total:.2f}".replace(".", ","))
        item_frame["total"].config(state="readonly")
    except ValueError:
        item_frame["total"].config(state="normal")
        item_frame["total"].delete(0, tk.END)
        item_frame["total"].insert(0, "0,00")
        item_frame["total"].config(state="readonly")
    calcular_total_cotacao()


def calcular_total_cotacao():
    total = 0
    for item_frame in item_frames:
        try:
            total += float(item_frame["total"].get().replace(",", "."))
        except ValueError:
            pass
    entrada_total_cotacao.config(state="normal")
    entrada_total_cotacao.delete(0, tk.END)
    entrada_total_cotacao.insert(0, f"{total:.2f}".replace(".", ","))
    entrada_total_cotacao.config(state="readonly")


# Gera a cotação ao clicar no botão
def gerar_cotacao():
    modo = cliente_mode_var.get()
    prazo_pagamento = entrada_prazo_pagamento.get()
    frete_tipo = frete_var.get()
    numero_sequencial = obter_numero_sequencial()
    responsavel_nome = responsavel_var.get()
    cliente_tipo = cliente_tipo_var.get()
    empresa = empresa_var.get()

    responsaveis = {
        "Weber Melo": {
            "nome": "Weber Melo",
            "telefone": "(11) 98477-9490",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br",
        },
        "Thiago Velicev": {
            "nome": "Thiago Velicev",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br",
        },
        "Giulia Armelin": {
            "nome": "Giulia Armelin",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br",
        },
        "Letícia Casale": {
            "nome": "Letícia Casale",
            "telefone": "(11) 91000-9205",
            "email": "atendimento@mokka-sensors.com.br",
            "site1": "www.mokka-sensors.com.br",
            "site2": "www.camerastermicas.com.br",
        },
    }

    responsavel = responsaveis.get(responsavel_nome)
    output_path = obter_caminho_saida(numero_sequencial)

    if not all([prazo_pagamento, frete_tipo, output_path, responsavel, cliente_tipo]):
        messagebox.showerror("Erro", "Todos os campos devem ser preenchidos.")
        return

    # Modo API (CNPJ) ou Manual
    if modo == "api":
        cnpj_cliente = entrada_cnpj.get().strip()
        if not cnpj_cliente:
            messagebox.showerror(
                "Erro",
                "Informe o CNPJ do cliente ou selecione 'Preenchimento manual'.",
            )
            return

        dados_cliente = buscar_dados_cliente(cnpj_cliente)
        if not dados_cliente:
            # Função de busca já mostra mensagem de erro
            return
    else:
        razao_manual = entrada_razao_manual.get().strip()
        endereco_manual = entrada_endereco_manual.get().strip()
        telefone_manual = entrada_telefone_manual.get().strip()

        if not all([razao_manual, endereco_manual, telefone_manual]):
            messagebox.showerror("Erro", "Preencha Nome, Endereço e Telefone do cliente.")
            return

        dados_cliente = {
            "razao_social": razao_manual,
            "endereco": endereco_manual,
            "cidade_uf_cep": "",
            "telefone": telefone_manual,
        }

    itens = []
    for item_frame in item_frames:
        produto = item_frame["produto"].get()
        descricao = item_frame["descricao"].get()
        preco_unitario = item_frame["preco_unitario"].get()
        quantidade = item_frame["quantidade"].get()
        ncm = item_frame["ncm"].get()
        total = item_frame["total"].get()
        prazo_entrega = item_frame["prazo_entrega"].get()

        if not all([produto, descricao, preco_unitario, quantidade, ncm, total, prazo_entrega]):
            messagebox.showerror("Erro", "Todos os campos dos itens devem ser preenchidos.")
            return

        itens.append(
            {
                "produto": produto,
                "descricao": descricao,
                "preco_unitario": preco_unitario,
                "quantidade": quantidade,
                "ncm": ncm,
                "total": total,
                "prazo_entrega": prazo_entrega,
            }
        )

    gerar_cotacao_pdf(
        dados_cliente,
        itens,
        prazo_pagamento,
        frete_tipo,
        output_path,
        numero_sequencial,
        responsavel,
        empresa,
    )
    adicionar_dados_excel(dados_cliente, itens, numero_sequencial, cliente_tipo)
    messagebox.showinfo("Sucesso", f"Cotação salva em: {output_path}")


# Funções para adicionar/remover itens
def adicionar_item():
    item_count = len(item_frames) + 1

    item_frame = {
        "label_item": tk.Label(items_frame, text=f"Item {item_count}"),
        "produto_label": tk.Label(items_frame, text="Produto:"),
        "produto": tk.Entry(items_frame, width=50),
        "descricao_label": tk.Label(items_frame, text="Descrição:"),
        "descricao": tk.Entry(items_frame, width=80),
        "preco_unitario_label": tk.Label(items_frame, text="Preço Unitário: R$"),
        "preco_unitario": tk.Entry(
            items_frame,
            width=20,
            validate="key",
            validatecommand=(janela.register(validate_decimal), "%P"),
        ),
        "quantidade_label": tk.Label(items_frame, text="Quantidade:"),
        "quantidade": tk.Entry(
            items_frame,
            width=20,
            validate="key",
            validatecommand=(janela.register(validate_integer), "%P"),
        ),
        "ncm_label": tk.Label(items_frame, text="NCM:"),
        "ncm": tk.Entry(items_frame, width=20),
        "total_label": tk.Label(items_frame, text="Total: R$"),
        "total": tk.Entry(items_frame, width=20, state="readonly"),
        "prazo_entrega_label": tk.Label(items_frame, text="Prazo de Entrega:"),
        "prazo_entrega": tk.Entry(items_frame, width=20),
        "botao_remover": None,
    }

    row = BASE_ITEM_ROW + len(item_frames) * 10

    item_frame["label_item"].grid(row=row, column=0, padx=10, pady=5, columnspan=2, sticky="w")
    item_frame["produto_label"].grid(row=row + 1, column=0, padx=10, pady=5, sticky="w")
    item_frame["produto"].grid(row=row + 1, column=1, padx=10, pady=5, sticky="w")
    item_frame["descricao_label"].grid(row=row + 2, column=0, padx=10, pady=5, sticky="w")
    item_frame["descricao"].grid(row=row + 2, column=1, padx=10, pady=5, sticky="w")
    item_frame["preco_unitario_label"].grid(row=row + 3, column=0, padx=10, pady=5, sticky="w")
    item_frame["preco_unitario"].grid(row=row + 3, column=1, padx=10, pady=5, sticky="w")
    item_frame["quantidade_label"].grid(row=row + 4, column=0, padx=10, pady=5, sticky="w")
    item_frame["quantidade"].grid(row=row + 4, column=1, padx=10, pady=5, sticky="w")
    item_frame["ncm_label"].grid(row=row + 5, column=0, padx=10, pady=5, sticky="w")
    item_frame["ncm"].grid(row=row + 5, column=1, padx=10, pady=5, sticky="w")
    item_frame["total_label"].grid(row=row + 6, column=0, padx=10, pady=5, sticky="w")
    item_frame["total"].grid(row=row + 6, column=1, padx=10, pady=5, sticky="w")
    item_frame["prazo_entrega_label"].grid(row=row + 7, column=0, padx=10, pady=5, sticky="w")
    item_frame["prazo_entrega"].grid(row=row + 7, column=1, padx=10, pady=5, sticky="w")

    item_frame["botao_remover"] = tk.Button(
        items_frame, text="Remover", command=lambda frame=item_frame: remover_item(frame)
    )
    item_frame["botao_remover"].grid(row=row + 8, column=0, columnspan=2, pady=5, sticky="w")

    item_frame["preco_unitario"].bind("<KeyRelease>", lambda event, frame=item_frame: calcular_total(frame))
    item_frame["quantidade"].bind("<KeyRelease>", lambda event, frame=item_frame: calcular_total(frame))

    item_frames.append(item_frame)
    items_frame.update_idletasks()
    tk_canvas.config(scrollregion=tk_canvas.bbox("all"))


def remover_item(item_frame):
    for widget in item_frame.values():
        if widget is not None:
            widget.grid_forget()
            widget.destroy()
    item_frames.remove(item_frame)
    recalcular_itens()


def recalcular_itens():
    for i, item_frame in enumerate(item_frames):
        base_row = BASE_ITEM_ROW + i * 10
        item_frame["label_item"].config(text=f"Item {i + 1}")
        item_frame["label_item"].grid(row=base_row, column=0, padx=10, pady=5, columnspan=2, sticky="w")
        item_frame["produto_label"].grid(row=base_row + 1, column=0, padx=10, pady=5, sticky="w")
        item_frame["produto"].grid(row=base_row + 1, column=1, padx=10, pady=5, sticky="w")
        item_frame["descricao_label"].grid(row=base_row + 2, column=0, padx=10, pady=5, sticky="w")
        item_frame["descricao"].grid(row=base_row + 2, column=1, padx=10, pady=5, sticky="w")
        item_frame["preco_unitario_label"].grid(row=base_row + 3, column=0, padx=10, pady=5, sticky="w")
        item_frame["preco_unitario"].grid(row=base_row + 3, column=1, padx=10, pady=5, sticky="w")
        item_frame["quantidade_label"].grid(row=base_row + 4, column=0, padx=10, pady=5, sticky="w")
        item_frame["quantidade"].grid(row=base_row + 4, column=1, padx=10, pady=5, sticky="w")
        item_frame["ncm_label"].grid(row=base_row + 5, column=0, padx=10, pady=5, sticky="w")
        item_frame["ncm"].grid(row=base_row + 5, column=1, padx=10, pady=5, sticky="w")
        item_frame["total_label"].grid(row=base_row + 6, column=0, padx=10, pady=5, sticky="w")
        item_frame["total"].grid(row=base_row + 6, column=1, padx=10, pady=5, sticky="w")
        item_frame["prazo_entrega_label"].grid(row=base_row + 7, column=0, padx=10, pady=5, sticky="w")
        item_frame["prazo_entrega"].grid(row=base_row + 7, column=1, padx=10, pady=5, sticky="w")
        item_frame["botao_remover"].grid(row=base_row + 8, column=0, columnspan=2, pady=5, sticky="w")

    calcular_total_cotacao()
    items_frame.update_idletasks()
    tk_canvas.config(scrollregion=tk_canvas.bbox("all"))


# Criação da interface gráfica
janela = tk.Tk()
janela.title("Gerador de Cotações")
janela.geometry("1200x800")

janela.grid_rowconfigure(0, weight=1)
janela.grid_columnconfigure(0, weight=1)

main_frame = tk.Frame(janela)
main_frame.grid(row=0, column=0, sticky="nsew")

tk_canvas = tk.Canvas(main_frame, width=1200, height=800)
tk_canvas.grid(row=0, column=0, sticky="nsew")

scrollbar = tk.Scrollbar(main_frame, orient="vertical", command=tk_canvas.yview)
scrollbar.grid(row=0, column=2, sticky="ns")

tk_canvas.configure(yscrollcommand=scrollbar.set)

items_frame = tk.Frame(tk_canvas, padx=10, pady=10)
tk_canvas.create_window((0, 0), window=items_frame, anchor="nw")


def _on_frame_configure(event):
    tk_canvas.configure(scrollregion=tk_canvas.bbox("all"))


items_frame.bind("<Configure>", _on_frame_configure)

# Seleção da empresa
tk.Label(items_frame, text="Empresa").grid(row=0, column=0, padx=10, pady=5, sticky="w")
empresa_var = tk.StringVar(value="Mokka")
empresa_options = ["Mokka", "Moica"]
empresa_menu = tk.OptionMenu(items_frame, empresa_var, *empresa_options)
empresa_menu.grid(row=0, column=1, padx=10, pady=5, sticky="w")

# Modo de preenchimento dos dados do cliente
tk.Label(items_frame, text="Modo de dados do cliente").grid(row=1, column=0, padx=10, pady=5, sticky="w")
cliente_mode_var = tk.StringVar(value="api")  # "api" (CNPJ) ou "manual"

modo_frame = tk.Frame(items_frame)
modo_frame.grid(row=1, column=1, padx=10, pady=5, sticky="w")

tk.Radiobutton(
    modo_frame, text="CNPJ (API ReceitaWS)", variable=cliente_mode_var, value="api"
).pack(side="left")
tk.Radiobutton(
    modo_frame, text="Preenchimento manual", variable=cliente_mode_var, value="manual"
).pack(side="left")

# Frame para CNPJ
cnpj_frame = tk.Frame(items_frame)
tk.Label(cnpj_frame, text="CNPJ do Cliente").grid(row=0, column=0, padx=10, pady=5, sticky="w")
entrada_cnpj = tk.Entry(cnpj_frame, width=50)
entrada_cnpj.grid(row=0, column=1, padx=10, pady=5, sticky="w")

# Frame para preenchimento manual (Nome, Endereço, Telefone)
manual_frame = tk.Frame(items_frame)
tk.Label(manual_frame, text="Nome / Razão Social").grid(row=0, column=0, padx=10, pady=5, sticky="w")
entrada_razao_manual = tk.Entry(manual_frame, width=50)
entrada_razao_manual.grid(row=0, column=1, padx=10, pady=5, sticky="w")

tk.Label(manual_frame, text="Endereço").grid(row=1, column=0, padx=10, pady=5, sticky="w")
entrada_endereco_manual = tk.Entry(manual_frame, width=50)
entrada_endereco_manual.grid(row=1, column=1, padx=10, pady=5, sticky="w")

tk.Label(manual_frame, text="Telefone").grid(row=2, column=0, padx=10, pady=5, sticky="w")
entrada_telefone_manual = tk.Entry(manual_frame, width=30)
entrada_telefone_manual.grid(row=2, column=1, padx=10, pady=5, sticky="w")


def atualizar_modo_cliente(*args):
    modo = cliente_mode_var.get()
    if modo == "api":
        manual_frame.grid_remove()
        cnpj_frame.grid(row=2, column=0, columnspan=2, padx=0, pady=0, sticky="w")
    else:
        cnpj_frame.grid_remove()
        manual_frame.grid(row=2, column=0, columnspan=2, padx=0, pady=0, sticky="w")


cliente_mode_var.trace_add("write", atualizar_modo_cliente)
# Estado inicial (API)
atualizar_modo_cliente()

# Responsável pela cotação
tk.Label(items_frame, text="Responsável pela Cotação").grid(row=3, column=0, padx=10, pady=5, sticky="w")
responsavel_options = ["Weber Melo", "Thiago Velicev", "Giulia Armelin", "Letícia Casale"]
responsavel_var = tk.StringVar(value=responsavel_options[0])
responsavel_menu = tk.OptionMenu(items_frame, responsavel_var, *responsavel_options)
responsavel_menu.grid(row=3, column=1, padx=10, pady=5, sticky="w")

# Tipo de cliente
tk.Label(items_frame, text="Tipo de Cliente").grid(row=4, column=0, padx=10, pady=5, sticky="w")
cliente_tipo_options = ["Cliente Final", "Revenda"]
cliente_tipo_var = tk.StringVar(value=cliente_tipo_options[0])
cliente_tipo_menu = tk.OptionMenu(items_frame, cliente_tipo_var, *cliente_tipo_options)
cliente_tipo_menu.grid(row=4, column=1, padx=10, pady=5, sticky="w")

item_frames = []
adicionar_item()

tk.Label(items_frame, text="TOTAL DA COTAÇÃO").grid(row=100, column=0, padx=10, pady=5, sticky="w")
entrada_total_cotacao = tk.Entry(items_frame, width=20, state="readonly")
entrada_total_cotacao.grid(row=100, column=1, padx=10, pady=5, sticky="w")

tk.Label(items_frame, text="Prazo de Pagamento").grid(row=101, column=0, padx=10, pady=5, sticky="w")
entrada_prazo_pagamento = tk.Entry(items_frame, width=20)
entrada_prazo_pagamento.grid(row=101, column=1, padx=10, pady=5, sticky="w")

tk.Label(items_frame, text="Frete").grid(row=102, column=0, padx=10, pady=5, sticky="w")
frete_var = tk.StringVar(value="FOB")
tk.Radiobutton(items_frame, text="FOB", variable=frete_var, value="FOB").grid(
    row=102, column=1, padx=10, pady=5, sticky="w"
)
tk.Radiobutton(items_frame, text="CIF", variable=frete_var, value="CIF").grid(
    row=102, column=1, padx=100, pady=5, sticky="w"
)

# Espaço entre campos e scrollbar
tk.Canvas(items_frame, width=cm).grid(row=0, column=3, rowspan=103)

botao_adicionar_item = tk.Button(items_frame, text="Adicionar Item", command=adicionar_item)
botao_adicionar_item.grid(row=104, column=0, columnspan=2, pady=10, sticky="w")

botao_gerar = tk.Button(items_frame, text="Gerar Cotação", command=gerar_cotacao)
botao_gerar.grid(row=105, column=0, columnspan=2, pady=20, sticky="w")

if __name__ == "__main__":
    janela.mainloop()


Diretório do script: C:\Users\weber\onedrive\jupyter\Codigo Fonte  Cotacoes Mokka Moica
